In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
X = df.drop('koi_disposition', axis=1)
y = df['koi_disposition']
print(X.shape, y.shape)

(6991, 40) (6991,)


# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale the Data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Label - encode data set 
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y=label_encoder.transform(y)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
encoded_y_test

array([1, 2, 2, ..., 2, 2, 0])

In [11]:
# Print the original and encoded data
for label, original_class in zip(y_test_categorical,  y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: CONFIRMED
Encoded Label: [0. 1. 0.]
------------
Original Class: FALSE POSITIVE
Encoded Label: [0. 0. 1.]
------------
Original Class: FALSE POSITIVE
Encoded Label: [0. 0. 1.]
------------
Original Class: CONFIRMED
Encoded Label: [0. 1. 0.]
------------
Original Class: CONFIRMED
Encoded Label: [0. 0. 1.]
------------
Original Class: CONFIRMED
Encoded Label: [0. 0. 1.]
------------
Original Class: CONFIRMED
Encoded Label: [0. 0. 1.]
------------
Original Class: CONFIRMED
Encoded Label: [0. 1. 0.]
------------
Original Class: CONFIRMED
Encoded Label: [1. 0. 0.]
------------
Original Class: CONFIRMED
Encoded Label: [0. 0. 1.]
------------
Original Class: CONFIRMED
Encoded Label: [0. 1. 0.]
------------
Original Class: FALSE POSITIVE
Encoded Label: [0. 0. 1.]
------------
Original Class: FALSE POSITIVE
Encoded Label: [0. 0. 1.]
------------
Original Class: FALSE POSITIVE
Encoded Label: [1. 0. 0.]
------------
Original Class: CONFIRMED
Encoded Label: [1. 0. 0.]
------------


# Train the Model



In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=70,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/70
5243/5243 - 0s - loss: 0.6287 - accuracy: 0.6813
Epoch 2/70
5243/5243 - 0s - loss: 0.3765 - accuracy: 0.8091
Epoch 3/70
5243/5243 - 0s - loss: 0.3610 - accuracy: 0.8203
Epoch 4/70
5243/5243 - 0s - loss: 0.3498 - accuracy: 0.8276
Epoch 5/70
5243/5243 - 0s - loss: 0.3413 - accuracy: 0.8350
Epoch 6/70
5243/5243 - 0s - loss: 0.3321 - accuracy: 0.8438
Epoch 7/70
5243/5243 - 0s - loss: 0.3327 - accuracy: 0.8459
Epoch 8/70
5243/5243 - 0s - loss: 0.3252 - accuracy: 0.8491
Epoch 9/70
5243/5243 - 0s - loss: 0.3199 - accuracy: 0.8501
Epoch 10/70
5243/5243 - 0s - loss: 0.3163 - accuracy: 0.8520
Epoch 11/70
5243/5243 - 0s - loss: 0.3132 - accuracy: 0.8610
Epoch 12/70
5243/5243 - 0s - loss: 0.3207 - accuracy: 0.8541
Epoch 13/70
5243/5243 - 0s - loss: 0.3059 - accuracy: 0.8617
Epoch 14/70
5243/5243 - 0s - loss: 0.3027 - accuracy: 0.8636
Epoch 15/70
5243/5243 - 0s - loss: 0.3023 - accuracy: 0.8623
Epoch 16/70
5243/5243 - 0s - loss: 0.3084 - accuracy: 0.8549
Epoch 17/70

# Quantify our Trained Model


In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.2342 - accuracy: 0.9073
Normal Neural Network - Loss: 0.23423790624812757, Accuracy: 0.9073226451873779


# Make Predictions

In [21]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [22]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CONFIRMED'
 'FALSE POSITIVE']
Actual Labels: ['CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'FALSE POSITIVE']


# Save the Model

In [26]:
model.save("Deeplearning_model.h5")